In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hotels_users_ratings.csv to hotels_users_ratings.csv


In [ ]:
df = pd.read_csv("hotels_users_ratings.csv", encoding="utf-8")

In [ ]:
print(df.columns.to_list())

['URL Hotel', 'Location', 'HotelID', 'Name Hotel', 'Descriptions', 'Address', 'UserID', 'User', 'Rating']


# # Bỏ cột UserID, User và URL

In [ ]:
df = df.drop(columns=["UserID", "User", "URL Hotel"])

In [ ]:
print(df.columns.to_list())

['Location', 'HotelID', 'Name Hotel', 'Descriptions', 'Address', 'Rating']


# # Kiểm tra dataset có miss dữ liệu ko

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38801 entries, 0 to 38800
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Location      38801 non-null  object
 1   HotelID       38801 non-null  int64 
 2   Name Hotel    38801 non-null  object
 3   Descriptions  38801 non-null  object
 4   Address       38801 non-null  object
 5   Rating        38801 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.8+ MB


# # Kiểm tra có dòng nào bị NaN không

In [ ]:
print(df.isnull().sum())

Location        0
HotelID         0
Name Hotel      0
Descriptions    0
Address         0
Rating          0
dtype: int64


# # Gộp theo HotelID

In [ ]:
df_grouped = df.groupby(
    ["HotelID", "Name Hotel", "Address", "Location"],
    as_index=False
).agg({
    "Descriptions": lambda x: " ".join(set(x)),
    "Rating": "mean"  # lấy trung bình
})

df_grouped["Rating"] = df_grouped["Rating"].round(1)

In [ ]:
# df_grouped = df_grouped.reset_index(drop=True)

In [ ]:
print(df_grouped.head(2))

   HotelID              Name Hotel  \
0        1  Bebe Homestay Phu Quoc   
1        2    Coucou Homestead - I   

                                             Address         Location  \
0  To 4 ap Ganh Gio Xa Cua Duong, Ong Lang, Phú Q...         Phú Quốc   
1  121/35 Lê Thị Riêng, Quận 1, TP. Hồ Chí Minh...  TP. Hồ Chí Minh   

                                        Descriptions  Rating  
0  Nằm trên đảo Phú Quốc, Bebe Homestay Phu Quoc ...     8.1  
1  Nằm cách Chợ ẩm thực đường phố Bến Thành chưa ...     7.0  


In [ ]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4533 entries, 0 to 4532
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HotelID       4533 non-null   int64  
 1   Name Hotel    4533 non-null   object 
 2   Address       4533 non-null   object 
 3   Location      4533 non-null   object 
 4   Descriptions  4533 non-null   object 
 5   Rating        4533 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 212.6+ KB


## test

In [ ]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 44.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=c4028c34517e517330f18f3f4aff7b6eacb711cb285bfae11debc54ab3b68188
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [ ]:
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0  # để kết quả ổn định

def is_english(text):
    try:
        return detect(text) == "en"
    except:
        return False

df_grouped["IsEnglish"] = df_grouped["Descriptions"].apply(is_english)
print(df_grouped["IsEnglish"].value_counts())

IsEnglish
False    3896
True      637
Name: count, dtype: int64


In [ ]:
# Xuất ra file CSV mới
df_grouped.to_csv("dataset_with_diacritics_flag.csv", index=False, encoding="utf-8-sig")

In [ ]:
# Giữ lại những dòng là tiếng việt
df_grouped = df_grouped[df_grouped["IsEnglish"] == False]

# Reset lại index cho đẹp
df_grouped = df_grouped.reset_index(drop=True)


In [ ]:
print(df_grouped["IsEnglish"].value_counts())

IsEnglish
False    3896
Name: count, dtype: int64


In [ ]:
df_grouped = df_grouped.drop(columns=["IsEnglish"])